In [1]:
import numpy as np
import cv2

In [2]:
# After the code is executed a VideoOutput.mp4 file will be visible in your directory.
# By tuning the parameters performance can be further improved.

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    ignore_mask_color = (255)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines):
  img = np.copy(img)
  blank_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

  for line in lines:
    for x1, y1, x2, y2 in line:
      cv2.line(blank_img, (x1,y1), (x2,y2), (0,0,255), thickness=2)
  
  img = cv2.addWeighted(img, 0.8, blank_img, 1, 0.0)
  return img

def process(image):
  def grayscale(image):
      return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  gray = grayscale(image)
  canny_image = cv2.Canny(gray, 100, 200)
  def gaussian_blur(img, kernel_size):
      """Applies a Gaussian Noise kernel"""
      return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
  gaus_image = gaussian_blur(canny_image, 1)

  height = image.shape[0]
  width = image.shape[1]
  vertices = [(0,height), (width/2, 310), (width, height)]
  masked = region_of_interest(gaus_image, np.array([vertices], np.int32))

  lines = cv2.HoughLinesP(masked,
                          rho=6,
                          theta=np.pi/60,
                          threshold=160,
                          lines=np.array([]),
                          minLineLength=40,
                          maxLineGap=25)

  image_with_lines = draw_lines(image, lines)
  return image_with_lines

cap = cv2.VideoCapture('Videos/Test.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('VideoOutput.mp4', fourcc, 20.0, (960,540))
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        Output = process(frame)
        out.write(Output)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()